<a href="https://colab.research.google.com/github/Hyeon-Kang/NLP/blob/master/week03_Naive_Bayes_model/NB_Grid%20search_stemming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 20개의 뉴스 카테고리 분류
# Naive Bayes 알고리즘을 통한 다항 분류
# Grid Search 를 사용하여 입력 범위내 최적의 파라미터 도출 

# Stemmer 전처리를 사용한 경우 Naive Bayes의 성능이 변하는지 실험
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset = 'train', shuffle=True)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer() # bag of word 형식으로 단어마다 인덱스를 부여한 벡터 생성
X_train_counts = count_vect.fit_transform(twenty_train.data)
print('Count Vector : ', X_train_counts.shape) # (문서의 개수, 단어의 개수)

Count Vector :  (11314, 130107)


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
# TF, IDF
tfidf_transformer = TfidfTransformer() # 단어 빈도와 문서 빈도를 통해 특정 문서 및 문서 집단에서의 단어 가중치 산출
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Tfidf : ', X_train_tfidf.shape) # (문서의 개수, 단어의 개수)

Tfidf :  (11314, 130107)


In [0]:
# 다항분류
print(' ---- Naive Bayes algorithm ---- ')

from sklearn.naive_bayes import MultinomialNB # 다항 분류 알고리즘

clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

from sklearn.pipeline import Pipeline

# 파이프라인을 통해 전처리 단계와 학습 모델을 결합한 구조체 작성
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())])

# 작성한 구조체에 데이터를 넣어 학습 진행
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

import numpy as np

twenty_test = fetch_20newsgroups(subset = 'test', shuffle = True) 
predicted = text_clf.predict(twenty_test.data) # 구축한 학습 모델 테스트
print('Accuracy : ', np.mean(predicted == twenty_test.target))





 ---- Naive Bayes algorithm ---- 
Accuracy :  0.7738980350504514


In [0]:
# Grid search를 통한 최적의 파라미터 경우 찾기
print(' ---- Grid serach : Naive Bayes algorithm ---- ')

from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range' : [(1,1), (1,2)], # 단어별로 학습, bigram 단위로 학습
             'tfidf__use_idf' : (True, False), # TF-IDF 가중치 사용 여부
             'clf__alpha' : (1e-2, 1e-3)} # 나이브 베이즈 다항분류의 페널티 정도 (0.01 or 0.001)

gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1) # -1 : 모든 cpu 사용
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print(gs_clf.best_score_)
print(gs_clf.best_params_) # 도출한 최적 파라미터

# print('Accuracy : ', np.mean(predicted == twenty_test.target))

 ---- Grid serach : Naive Bayes algorithm ---- 


c:\python\venv\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.9067526957751458
{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


In [0]:
# stemmer를 사용하여 단어의 원형으로 바꾼 뒤  Naive bayes 알고리즘을 통해 분류한다면
# 더 좋은 효율이 나올까?
print('---- Naive bayes with Stemming ----')

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer() # stemming을 한 뒤 CountVectorize 적용
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), # stemming된 데이터를 CountVetorizer 적용
                            ('tfidf', TfidfTransformer()),
                            ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(twenty_train.data, twenty_train.target)
predicted_mnb_stemmed = text_mnb_stemmed.predict(twenty_test.data)
print('Accuracy : ', np.mean(predicted_mnb_stemmed == twenty_test.target))

# 더 좋은 데이터가 나옴을 확인

---- Naive bayes with Stemming ----
Accuracy :  0.8167817312798725
